In [40]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL Project")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

sdf = spark.read.parquet("../data/curated/process_data.parquet/")

cols = ['merchant_abn', 'user_id', 'dollar_value', 'order_datetime', 'rate', 'category', 'subcategory', 'estimated_region_population_2021_sum',	'persons_earners_2018-19_sum', 'mean_earnings_2018-19_avg', 'sum_earnings_2018-19_sum',	'median_earnings_2018-19_avg', 'med_age_earners_2018-19_avg']
sdf = sdf.select(cols)

sdf = sdf.withColumn('rate', F.col('rate') / 100)

sdf.limit(10)

In [14]:
cols = ['merchant_abn', 'user_id', 'dollar_value', 'order_datetime', 'rate', 'category', 'subcategory', 'estimated_region_population_2021_sum',	'persons_earners_2018-19_sum', 'mean_earnings_2018-19_avg', 'sum_earnings_2018-19_sum',	'median_earnings_2018-19_avg', 'med_age_earners_2018-19_avg']
sdf = sdf.select(cols)

sdf = sdf.withColumn('rate', F.col('rate') / 100)

sdf.limit(10)

merchant_abn,user_id,dollar_value,order_datetime,rate,category,subcategory,estimated_region_population_2021_sum,persons_earners_2018-19_sum,mean_earnings_2018-19_avg,sum_earnings_2018-19_sum,median_earnings_2018-19_avg,med_age_earners_2018-19_avg
27093785141,1,366.23,2021-11-17,0.0273,retail_and_wholes...,others_retailing,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
49125619545,1,113.59,2022-06-28,0.056600000000000004,info_media_and_te...,null,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
28510201393,1,587.4,2021-09-24,0.018600000000000002,retail_and_wholes...,department_stores,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
49167531725,1,51.58,2021-11-26,0.0642,retail_and_wholes...,household_goods_r...,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
43719937438,1,196.41,2022-05-22,0.0501,info_media_and_te...,null,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
49278214714,1,7.84,2022-08-10,0.0599,retail_and_wholes...,household_goods_r...,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
46007761675,1,100.22,2022-10-24,0.0572,retail_and_wholes...,household_goods_r...,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
50315283629,1,115.82,2021-11-13,0.0176,retail_and_wholes...,food_retailing,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
46654841462,1,18.5,2022-09-03,0.0381,rental_hiring_and...,null,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
50315283629,1,63.88,2022-10-13,0.0176,retail_and_wholes...,food_retailing,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0


### 1. **Identify** Merchants with the greatest gross and lowest volatility in income

In [39]:
sdf.groupBy('merchant_abn').agg(
        F.sum('dollar_value').alias('income_total'),
        F.stddev('dollar_value').alias('income_deviation'),
        F.mean('rate').alias('rate'),
        F.count('merchant_abn').alias('count_merchant'))\
    .withColumn('income_total', F.col('income_total') * F.col('rate'))\
    \
    .orderBy(F.col('count_merchant').desc(), F.col('income_total').desc(), F.col('income_deviation').asc())

merchant_abn,income_total,income_deviation,rate,count_merchant
24852446429,255744.12288606438,21.178693915107022,0.02940000000000741,289513
86578477987,613410.4864420284,24.776256575329793,0.06430000000000283,272674
64203420245,215914.12099394575,15.025235382946878,0.028599999999992777,260780
49891706470,415952.53725995746,15.014902745150149,0.057999999999993966,247526
46804135891,206349.62331796187,30.04859263484707,0.02929999999999459,234397
45629217853,585975.3666878993,36.721188382863424,0.06979999999998802,228219
89726005175,535420.1324840862,23.79102051654588,0.0601000000000097,215963
43186523025,404176.7518500293,25.948689819226168,0.04470000000000324,200913
80324045558,106133.55576602292,36.808440403202795,0.01470000000000318,196816
63290521567,285260.53175996797,24.296439621551322,0.06479999999999268,181841


In [30]:
sdf.where(F.col('merchant_abn') == 79827781481).count()

4798